In [1]:
import torch
import numpy as np

In [2]:
#tensor initialization <directly from data>
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

#tensor initialization <from np array>
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

print(x_data)
print(x_np)

print("\n" + str(x_np.shape))
print(f"type is {x_data.dtype} vs {x_np.dtype}")
print(f"tensor is stored on {x_np.device}")

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])

torch.Size([2, 2])
type is torch.int64 vs torch.int64
tensor is stored on cpu


In [3]:
x_ones = torch.ones_like(x_data)
print(f"ones tensor: \n {x_ones}")

x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"random tensor: \n {x_ones}")

ones tensor: 
 tensor([[1, 1],
        [1, 1]])
random tensor: 
 tensor([[1, 1],
        [1, 1]])


In [4]:
shape = (3,5)
rand_tensor = torch.rand(shape)
print(f"random tensor: \n {rand_tensor}")

random tensor: 
 tensor([[0.5082, 0.4485, 0.9666, 0.8764, 0.2403],
        [0.5017, 0.7652, 0.3138, 0.8634, 0.9543],
        [0.2845, 0.7273, 0.2784, 0.9439, 0.8410]])


In [5]:
tensor = torch.ones(3,4)
tensor2 = torch.rand(3,4)

if torch.cuda.is_available():
  tensor = tensor.to('cuda')

print(tensor)
print(f"Device tensor is stored on: {tensor.device}")

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
Device tensor is stored on: cpu


In [6]:
tensor[:,1]=0
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [7]:
t1 = torch.cat([tensor, tensor], dim = 1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.]])


In [8]:
print(tensor.mul(tensor))

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [9]:
print(tensor.matmul(tensor.T))

tensor([[3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.]])


In [10]:
tensor.add_(3)
print(tensor)

tensor([[4., 3., 4., 4.],
        [4., 3., 4., 4.],
        [4., 3., 4., 4.]])


In [11]:
#Torch AUTOGRAD
import torchvision

In [12]:
model = torchvision.models.resnet18()
data = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
predictions = model(data)
print(predictions.size())

torch.Size([1, 1000])


In [14]:
loss = (predictions - labels).sum()
print(loss.detach().numpy())

-502.503


In [15]:
loss.backward()

In [16]:
optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [17]:
optim.zero_grad()

In [18]:
optim.step()

In [19]:
#now onto a simple neural network

import torch.nn as nn
import torch.nn.functional as F

In [20]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.Conv1 = nn.Conv2d(1, 6, 5)
        self.Conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # first, max pool over 2x2 window
        x = F.max_pool2d(F.relu(self.Conv1(x)), (2,2))
        # same thing here, short hand operation for 2x2 square
        x = F.max_pool2d(F.relu(self.Conv2(x)), 2)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
net = Net()
print(net)

Net(
  (Conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (Conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [22]:
params = list(net.parameters())
#print(params)
print(len(params))
#print(params[0].size())  # conv1's .weight
for i in range(len(params)):
    print(params[i].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [23]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0955, -0.0773,  0.0259, -0.0768,  0.0715, -0.0223,  0.0320, -0.0218,
          0.0309,  0.0436]], grad_fn=<AddmmBackward0>)


In [24]:
# net.zero_grad()
# out.backward(torch.randn(1, 10))

In [25]:
target = torch.randn(10)
target = target.view(1, -1)
criteron = nn.MSELoss()

loss = criteron(out, target)
print(loss)

tensor(0.4129, grad_fn=<MseLossBackward0>)


In [26]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions) #linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #relu

((<AddmmBackward0 object at 0x7ff453717550>, 0), (None, 0))


In [27]:
net.zero_grad()

In [28]:
print('conv1.bias.grad before backward')
print(net.Conv1.bias.grad)

conv1.bias.grad before backward
None


In [29]:
loss.backward()

In [30]:
print('conv1.bias.grad after backward')
print(net.Conv1.bias.grad)

conv1.bias.grad after backward
tensor([-0.0030, -0.0029,  0.0023,  0.0102, -0.0003,  0.0037])


In [31]:
learning_rate = 0.01
# weight = weight - learning rate * gradient
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [32]:
import torch.optim as optimizer

opt = optimizer.SGD(net.parameters(), lr=0.01)

opt.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
opt.s